In [1]:
import cv2
import mediapipe as mp

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
data_path="../../data/sample_videos/sample1.mp4"
_, vid_file_name = os.path.split(data_path)

vid_file_name

'sample1.mp4'

In [11]:
video_summary={}

video_summary["filename"]=vid_file_name

In [6]:
video = cv2.VideoCapture(data_path)

get_pose_details(data_path, True)

NameError: name 'mp' is not defined

In [5]:
def get_pose_details(file_path, visualize):
    """This function gets pose details from mediapipe

    Args:
        file_path (string): This is the path of the video file
        visualize (bool, optional): Provides a visualization for the pose tracking. Defaults to False.

    Returns:
        dict: This dictionary contains a list of upper body pose estimates
    """
    cap = cv2.VideoCapture(file_path)
    mp_pose = mp.solutions.pose

    pose_x = []
    pose_y = []

    with mp_pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_height, image_width, _ = image.shape

            pose_results = pose.process(image)

            pose_x_frame = []
            pose_y_frame = []

            for i in range(23):
                pose_x_frame.append(
                    round(pose_results.pose_landmarks.landmark[i].x * image_width, 2)
                )
                pose_y_frame.append(
                    round(pose_results.pose_landmarks.landmark[i].y * image_height, 2)
                )
            # print(pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image_width)

            pose_x.append(pose_x_frame)
            pose_y.append(pose_y_frame)

            # visualizing the pose analysis
            if visualize:
                mp_drawing = mp.solutions.drawing_utils
                mp_drawing_styles = mp.solutions.drawing_styles
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image,
                    pose_results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
                )
                cv2.imshow("MediaPipe Pose", cv2.flip(image, 1))
                if cv2.waitKey(5) & 0xFF == 27:
                    break

    cap.release()
